In [52]:
import pandas as pd

crop = pd.read_csv('./csv/crop_growing_conditions.csv', encoding='cp949')

weather_prod = pd.read_csv('./csv/final_merged_production_weather_data_new.csv')

weather=pd.read_csv('./csv/weather_area_new.csv')

weather

,TM,STN,WS_AVG,WR_DAY,WD_MAX,WS_MAX,WS_MAX_TM,WD_INS,WS_INS,WS_INS_TM,...,SD_MAX,SD_MAX_TM,TE_05,TE_10,TE_15,TE_30,TE_50,STN_ID,STN_KO,AREA
0,20140101,90.0,5.4,4691.0,27.0,9.8,2224.0,27.0,18.0,2111.0,...,-9.0,-9.0,-99.0,-99.0,-99.0,-99.0,-99.0,90.0,속초,강원도
1,20140101,95.0,3.2,2741.0,23.0,7.3,1527.0,23.0,12.5,1611.0,...,0.5,1.0,2.2,5.6,-99.0,-99.0,-99.0,95.0,철원,강원도
2,20140101,98.0,2.6,2257.0,25.0,7.9,1541.0,25.0,13.7,1259.0,...,-9.0,-9.0,-99.0,-99.0,-99.0,-99.0,-99.0,98.0,동두천,경기도
3,20140101,99.0,2.0,1742.0,23.0,6.1,1442.0,25.0,10.0,1521.0,...,-9.0,-9.0,-99.0,-99.0,-99.0,-99.0,-99.0,99.0,파주,경기도
4,20140101,100.0,10.6,9158.0,27.0,15.5,1837.0,27.0,22.7,1935.0,...,2.4,0.0,-99.0,-99.0,-99.0,-99.0,-99.0,100.0,대관령,강원도
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359833,20240615,285.0,0.7,615.0,18.0,3.8,1832.0,16.0,8.2,1830.0,...,-9.0,-9.0,-99.0,-99.0,-99.0,-99.0,-99.0,285.0,합천,경상남도
359834,20240615,288.0,0.8,728.0,23.0,2.9,1637.0,36.0,5.7,1855.0,...,-9.0,-9.0,-99.0,-99.0,-99.0,-99.0,-99.0,288.0,밀양,경상남도
359835,20240615,289.0,1.2,1003.0,29.0,3.9,1916.0,32.0,6.9,1942.0,...,-9.0,-9.0,-99.0,-99.0,-99.0,-99.0,-99.0,289.0,산청,경상남도
359836,20240615,294.0,1.3,1118.0,20.0,4.4,2011.0,14.0,10.0,2007.0,...,-9.0,-9.0,-99.0,-99.0,-99.0,-99.0,-99.0,294.0,거제,경상남도


In [3]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

## 날씨 데이터를 작물별로 달에 맞춰서 자르기

In [34]:
crop['파종기'] = pd.to_numeric(crop['파종기'], errors='coerce')
crop['수확기'] = pd.to_numeric(crop['수확기'], errors='coerce')

filtered_data = pd.DataFrame()

for index, row in crop.iterrows():
    crop = row['작물명']
    region = row['지역']
    planting_month = row['파종기']
    harvest_month = row['수확기']
    
    # Filter the final_merged_production_weather_data based on the crop and region
    filtered = weather_prod[
        (weather_prod['작물명'] == crop) & 
        (weather_prod['지역'] == region)
    ]
    
    # Further filter the data based on the planting and harvest months
    filtered = filtered[
        (filtered['MONTH'] >= planting_month) & 
        (filtered['MONTH'] <= harvest_month)
    ]

filtered_data = pd.concat([filtered_data, filtered])

In [35]:
filtered_data

,YEAR,MONTH,지역,작물명,생산량,RN_DAY,WS_AVG,PA_AVG,SS_DAY,HM_AVG,TA_AVG
8,2014.0,9.0,충청남도,가을일반무,76430.0,-2.437333,1.198667,1007.551333,6.559333,78.574000,20.581333
9,2014.0,10.0,충청남도,가을일반무,76430.0,-2.307097,1.530968,1013.685806,7.192258,74.468387,14.285806
10,2014.0,11.0,충청남도,가을일반무,76430.0,-3.879333,1.416000,1015.718000,5.134667,76.842000,7.964000
20,2015.0,9.0,충청남도,가을일반무,123235.0,-6.306667,1.356667,1008.239333,7.102667,75.099333,20.526667
21,2015.0,10.0,충청남도,가을일반무,123235.0,-3.615484,1.340000,1012.965161,7.054839,77.261935,14.465161
...,...,...,...,...,...,...,...,...,...,...,...
8847,2021.0,8.0,전라남도,호박,33804.0,5.461042,1.915385,999.828040,5.803226,83.829280,25.666501
8856,2022.0,5.0,전라남도,호박,45586.0,-7.508750,2.272250,1007.048000,9.033750,63.449000,17.256500
8857,2022.0,6.0,전라남도,호박,45586.0,-1.602821,2.675641,999.391795,5.260256,80.731795,22.032308
8858,2022.0,7.0,전라남도,호박,45586.0,2.309181,2.424566,995.163275,6.081390,82.142432,25.726303


In [36]:
filtered_data[filtered_data['작물명']=='당근']

,YEAR,MONTH,지역,작물명,생산량,RN_DAY,WS_AVG,PA_AVG,SS_DAY,HM_AVG,TA_AVG
3008,2014.0,3.0,제주도,당근,57486.0,-0.830645,4.399194,1015.129839,6.315323,66.831452,10.684677
3009,2014.0,4.0,제주도,당근,57486.0,-1.662500,3.618333,1011.915833,6.515833,67.318333,14.676667
3010,2014.0,5.0,제주도,당근,57486.0,-0.736290,3.327419,1007.417742,8.741935,70.157258,18.480645
3011,2014.0,6.0,제주도,당근,57486.0,0.027500,2.893333,1001.840000,5.021667,84.295833,21.244167
3012,2014.0,7.0,제주도,당근,57486.0,7.845968,3.027419,1002.851613,4.046774,87.027419,24.770968
3020,2015.0,3.0,제주도,당근,62814.0,-1.262903,3.525806,1016.434677,6.519355,64.954839,10.074194
3021,2015.0,4.0,제주도,당근,62814.0,3.181667,3.485833,1010.925833,5.565000,76.488333,14.320000
3022,2015.0,5.0,제주도,당근,62814.0,3.632258,3.059677,1005.791935,7.941129,72.270968,17.207258
3023,2015.0,6.0,제주도,당근,62814.0,3.859167,2.632500,1002.537500,4.623333,85.615833,20.945000
3024,2015.0,7.0,제주도,당근,62814.0,4.985484,3.363710,1002.416935,5.733065,87.750000,24.373387


In [37]:
filtered_data['YEAR'] = filtered_data['YEAR'].astype(int)
filtered_data['MONTH'] = filtered_data['MONTH'].astype(int)

In [5]:
weather_prod['지역'].unique()

array(['충청남도', '경기도', '경상북도', '강원도', '경상남도', '제주도', '전라남도', '전라북도'],
      dtype=object)

## 날씨 예측 (SARIMA 모델)
### 1. 충청남도

In [7]:
충청남도_weather = weather[weather['AREA'] == '충청남도']

충청남도_weather['DATE'] = pd.to_datetime(충청남도_weather['TM'], format='%Y%m%d')

충청남도_weather.set_index('DATE', inplace=True)

monthly_avg_temp = 충청남도_weather['TA_AVG'].resample('M').mean().fillna(method='ffill')

충청남도_model=SARIMAX(monthly_avg_temp, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))

sarima_result = 충청남도_model.fit(disp=False)

sarima_forecast_steps = 3
sarima_temperature_forecast = sarima_result.forecast(steps=sarima_forecast_steps)

sarima_forecast_dates = pd.date_range(start=monthly_avg_temp.index[-1], periods=sarima_forecast_steps + 1, freq='M')[1:]
충청남도_pred = pd.DataFrame({'DATE': sarima_forecast_dates, 'PREDICTED_TA_AVG': sarima_temperature_forecast.values})

충청남도_pred

C:\Users\sunk2\AppData\Local\Temp\ipykernel_19276\4192031006.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  충청남도_weather['DATE'] = pd.to_datetime(충청남도_weather['TM'], format='%Y%m%d')


,DATE,PREDICTED_TA_AVG
0,2024-07-31,25.405726
1,2024-08-31,26.089357
2,2024-09-30,21.488450


### 경기도

In [8]:
경기도_weather = weather[weather['AREA'] == '경기도']

경기도_weather['DATE'] = pd.to_datetime(경기도_weather['TM'], format='%Y%m%d')

경기도_weather.set_index('DATE', inplace=True)

monthly_avg_temp = 경기도_weather['TA_AVG'].resample('M').mean().fillna(method='ffill')

경기도_model=SARIMAX(monthly_avg_temp, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))

sarima_result = 경기도_model.fit(disp=False)

sarima_forecast_steps = 3
sarima_temperature_forecast = sarima_result.forecast(steps=sarima_forecast_steps)

sarima_forecast_dates = pd.date_range(start=monthly_avg_temp.index[-1], periods=sarima_forecast_steps + 1, freq='M')[1:]
경기도_pred = pd.DataFrame({'DATE': sarima_forecast_dates, 'PREDICTED_TA_AVG': sarima_temperature_forecast.values})

경기도_pred

C:\Users\sunk2\AppData\Local\Temp\ipykernel_19276\740547673.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  경기도_weather['DATE'] = pd.to_datetime(경기도_weather['TM'], format='%Y%m%d')


,DATE,PREDICTED_TA_AVG
0,2024-07-31,25.619428
1,2024-08-31,25.790288
2,2024-09-30,20.735910


### 경상북도

In [10]:
경상북_weather = weather[weather['AREA'] == '경상북도']

경상북_weather['DATE'] = pd.to_datetime(경상북_weather['TM'], format='%Y%m%d')

경상북_weather.set_index('DATE', inplace=True)

monthly_avg_temp = 경상북_weather['TA_AVG'].resample('M').mean().fillna(method='ffill')

경상북_model=SARIMAX(monthly_avg_temp, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))

sarima_result = 경상북_model.fit(disp=False)

sarima_forecast_steps = 3
sarima_temperature_forecast = sarima_result.forecast(steps=sarima_forecast_steps)

sarima_forecast_dates = pd.date_range(start=monthly_avg_temp.index[-1], periods=sarima_forecast_steps + 1, freq='M')[1:]
경상북_pred = pd.DataFrame({'DATE': sarima_forecast_dates, 'PREDICTED_TA_AVG': sarima_temperature_forecast.values})

경상북_pred

C:\Users\sunk2\AppData\Local\Temp\ipykernel_19276\103282134.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  경상북_weather['DATE'] = pd.to_datetime(경상북_weather['TM'], format='%Y%m%d')


,DATE,PREDICTED_TA_AVG
0,2024-07-31,25.047863
1,2024-08-31,25.339932
2,2024-09-30,20.500862


### 강원도

In [11]:
강원_weather = weather[weather['AREA'] == '강원도']

강원_weather['DATE'] = pd.to_datetime(강원_weather['TM'], format='%Y%m%d')

강원_weather.set_index('DATE', inplace=True)

monthly_avg_temp = 강원_weather['TA_AVG'].resample('M').mean().fillna(method='ffill')

강원_model=SARIMAX(monthly_avg_temp, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))

sarima_result = 강원_model.fit(disp=False)

sarima_forecast_steps = 3
sarima_temperature_forecast = sarima_result.forecast(steps=sarima_forecast_steps)

sarima_forecast_dates = pd.date_range(start=monthly_avg_temp.index[-1], periods=sarima_forecast_steps + 1, freq='M')[1:]
강원_pred = pd.DataFrame({'DATE': sarima_forecast_dates, 'PREDICTED_TA_AVG': sarima_temperature_forecast.values})

강원_pred

C:\Users\sunk2\AppData\Local\Temp\ipykernel_19276\1530407387.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  강원_weather['DATE'] = pd.to_datetime(강원_weather['TM'], format='%Y%m%d')


,DATE,PREDICTED_TA_AVG
0,2024-07-31,24.348255
1,2024-08-31,24.363202
2,2024-09-30,19.384474


### 경상남도

In [12]:
경상남_weather = weather[weather['AREA'] == '경상남도']

경상남_weather['DATE'] = pd.to_datetime(경상남_weather['TM'], format='%Y%m%d')

경상남_weather.set_index('DATE', inplace=True)

monthly_avg_temp = 경상남_weather['TA_AVG'].resample('M').mean().fillna(method='ffill')

경상남_model=SARIMAX(monthly_avg_temp, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))

sarima_result = 경상남_model.fit(disp=False)

sarima_forecast_steps = 3
sarima_temperature_forecast = sarima_result.forecast(steps=sarima_forecast_steps)

sarima_forecast_dates = pd.date_range(start=monthly_avg_temp.index[-1], periods=sarima_forecast_steps + 1, freq='M')[1:]
경상남_pred = pd.DataFrame({'DATE': sarima_forecast_dates, 'PREDICTED_TA_AVG': sarima_temperature_forecast.values})

경상남_pred

C:\Users\sunk2\AppData\Local\Temp\ipykernel_19276\1186592753.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  경상남_weather['DATE'] = pd.to_datetime(경상남_weather['TM'], format='%Y%m%d')


,DATE,PREDICTED_TA_AVG
0,2024-07-31,25.641932
1,2024-08-31,26.447722
2,2024-09-30,21.878726


### 제주도

In [18]:
제주_weather = weather[weather['AREA'] == '제주특별자치도']

제주_weather['DATE'] = pd.to_datetime(제주_weather['TM'], format='%Y%m%d')

제주_weather.set_index('DATE', inplace=True)

monthly_avg_temp = 제주_weather['TA_AVG'].resample('M').mean().fillna(method='ffill')

제주_model=SARIMAX(monthly_avg_temp, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))

sarima_result = 제주_model.fit(disp=False)

sarima_forecast_steps = 3
sarima_temperature_forecast = sarima_result.forecast(steps=sarima_forecast_steps)

sarima_forecast_dates = pd.date_range(start=monthly_avg_temp.index[-1], periods=sarima_forecast_steps + 1, freq='M')[1:]
제주_pred = pd.DataFrame({'DATE': sarima_forecast_dates, 'PREDICTED_TA_AVG': sarima_temperature_forecast.values})

제주_pred

C:\Users\sunk2\AppData\Local\Temp\ipykernel_19276\151455144.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  제주_weather['DATE'] = pd.to_datetime(제주_weather['TM'], format='%Y%m%d')


,DATE,PREDICTED_TA_AVG
0,2024-07-31,26.012078
1,2024-08-31,27.761229
2,2024-09-30,24.296946


### 전라남도

In [15]:
전라남_weather = weather[weather['AREA'] == '전라남도']

전라남_weather['DATE'] = pd.to_datetime(전라남_weather['TM'], format='%Y%m%d')

전라남_weather.set_index('DATE', inplace=True)

monthly_avg_temp = 전라남_weather['TA_AVG'].resample('M').mean().fillna(method='ffill')

전라남_model=SARIMAX(monthly_avg_temp, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))

sarima_result = 전라남_model.fit(disp=False)

sarima_forecast_steps = 3
sarima_temperature_forecast = sarima_result.forecast(steps=sarima_forecast_steps)

sarima_forecast_dates = pd.date_range(start=monthly_avg_temp.index[-1], periods=sarima_forecast_steps + 1, freq='M')[1:]
전라남_pred = pd.DataFrame({'DATE': sarima_forecast_dates, 'PREDICTED_TA_AVG': sarima_temperature_forecast.values})

전라남_pred

C:\Users\sunk2\AppData\Local\Temp\ipykernel_19276\2683509229.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  전라남_weather['DATE'] = pd.to_datetime(전라남_weather['TM'], format='%Y%m%d')


,DATE,PREDICTED_TA_AVG
0,2024-07-31,24.835428
1,2024-08-31,26.250957
2,2024-09-30,22.246643


### 전라북도

In [16]:
전라북_weather = weather[weather['AREA'] == '전라북도']

전라북_weather['DATE'] = pd.to_datetime(전라북_weather['TM'], format='%Y%m%d')

전라북_weather.set_index('DATE', inplace=True)

monthly_avg_temp = 전라북_weather['TA_AVG'].resample('M').mean().fillna(method='ffill')

전라북_model=SARIMAX(monthly_avg_temp, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))

sarima_result = 전라북_model.fit(disp=False)

sarima_forecast_steps = 3
sarima_temperature_forecast = sarima_result.forecast(steps=sarima_forecast_steps)

sarima_forecast_dates = pd.date_range(start=monthly_avg_temp.index[-1], periods=sarima_forecast_steps + 1, freq='M')[1:]
전라북_pred = pd.DataFrame({'DATE': sarima_forecast_dates, 'PREDICTED_TA_AVG': sarima_temperature_forecast.values})

전라북_pred

C:\Users\sunk2\AppData\Local\Temp\ipykernel_19276\3853302773.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  전라북_weather['DATE'] = pd.to_datetime(전라북_weather['TM'], format='%Y%m%d')


,DATE,PREDICTED_TA_AVG
0,2024-07-31,25.517415
1,2024-08-31,26.198830
2,2024-09-30,21.516736


## 작물 추천 여부

In [111]:
충청남_crops = crop[crop['지역'] == '충청남도']

전체_suit=[]

def check_crop_suitability_sarima(forecast_temp, forecast_date, crop_conditions):
    suitable_crops = []
    reasons = {}
    forecast_month = forecast_date.month
    for _, row in crop_conditions.iterrows():
        crop_name = row['작물명']
        planting_month = row['파종기']
        harvesting_month = row['수확기']
        
        reasons[crop_name] = []
        
        if not (planting_month -1 <= forecast_month <= harvesting_month +1):
            reasons[crop_name].append(f"Not within planting ({planting_month}) and harvesting ({harvesting_month}) period.")
        
        if not (row['생육기 최저 온도']-1 <= forecast_temp <= row['생육기 최고 온도']+1):
            reasons[crop_name].append(f"Temperature {forecast_temp:.2f}°C not in growing range ({row['생육기 최저 온도']}°C - {row['생육기 최고 온도']}°C).")
        
        if not reasons[crop_name]:
            suitable_crops.append(crop_name)
        else:
            reasons[crop_name] = ", ".join(reasons[crop_name])
    
    return suitable_crops, reasons

충청남_suit = []
reasons_forecast = []

for i in range(len(sarima_forecasted_temp_df)):
    forecast_temp = sarima_temperature_forecast[i]
    forecast_date = sarima_forecasted_temp_df['DATE'][i]
    suitable_crops, reasons = check_crop_suitability_sarima(forecast_temp, forecast_date, 충청남_crops)
    suitable_crops_all, reasons = check_crop_suitability_sarima(forecast_temp, forecast_date, 충청남_crops)
    충청남_suit.append(suitable_crops)
    전체_suit.append(suitable_crops_all)
    reasons_forecast.append(reasons)

충청남_suit


[['노지참외', '수박', '오이'], ['노지참외', '수박'], ['가을일반무', '노지가을무', '노지참외', '수박']]

In [112]:
경기_crops = crop[crop['지역'] == '경기도']

경기_suit = []
reasons_forecast = []

for i in range(len(sarima_forecasted_temp_df)):
    forecast_temp = sarima_temperature_forecast[i]
    forecast_date = sarima_forecasted_temp_df['DATE'][i]
    suitable_crops, reasons = check_crop_suitability_sarima(forecast_temp, forecast_date, 경기_crops)
    경기_suit.append(suitable_crops)
    suitable_crops_all, reasons = check_crop_suitability_sarima(forecast_temp, forecast_date, 경기_crops)
    전체_suit.append(suitable_crops_all)
    reasons_forecast.append(reasons)

경기_suit


[[], [], ['가을총각무', '상추']]

In [113]:
경상북_crops = crop[crop['지역'] == '경상북도']

경상북_suit = []
reasons_forecast = []

for i in range(len(sarima_forecasted_temp_df)):
    forecast_temp = sarima_temperature_forecast[i]
    forecast_date = sarima_forecasted_temp_df['DATE'][i]
    suitable_crops, reasons = check_crop_suitability_sarima(forecast_temp, forecast_date, 경상북_crops)
    경상북_suit.append(suitable_crops)
    suitable_crops_all, reasons = check_crop_suitability_sarima(forecast_temp, forecast_date, 경상북_crops)
    전체_suit.append(suitable_crops_all)
    reasons_forecast.append(reasons)

경상북_suit


[['건고추', '노지수박', '생강', '참외'],
 ['건고추', '노지수박', '생강'],
 ['건고추', '노지수박', '사과', '생강']]

In [114]:
강원_crops = crop[crop['지역'] == '강원도']

강원_suit = []
reasons_forecast = []

for i in range(len(sarima_forecasted_temp_df)):
    forecast_temp = sarima_temperature_forecast[i]
    forecast_date = sarima_forecasted_temp_df['DATE'][i]
    suitable_crops, reasons = check_crop_suitability_sarima(forecast_temp, forecast_date, 강원_crops)
    강원_suit.append(suitable_crops)
    suitable_crops_all, reasons = check_crop_suitability_sarima(forecast_temp, forecast_date, 강원_crops)
    전체_suit.append(suitable_crops_all)
    reasons_forecast.append(reasons)

강원_suit


[['노지오이', '옥수수'], ['노지오이', '옥수수'], ['옥수수', '잡곡']]

In [115]:
경상남_crops = crop[crop['지역'] == '경상남도']

경상남_suit = []
reasons_forecast = []

for i in range(len(sarima_forecasted_temp_df)):
    forecast_temp = sarima_temperature_forecast[i]
    forecast_date = sarima_forecasted_temp_df['DATE'][i]
    suitable_crops, reasons = check_crop_suitability_sarima(forecast_temp, forecast_date, 경상남_crops)
    경상남_suit.append(suitable_crops)
    suitable_crops_all, reasons = check_crop_suitability_sarima(forecast_temp, forecast_date, 경상남_crops)
    전체_suit.append(suitable_crops_all)
    reasons_forecast.append(reasons)

경상남_suit


[['고추', '풋고추'], ['고추', '풋고추'], ['고추', '노지시금치']]

In [116]:
제주_crops = crop[crop['지역'] == '제주도']

제주_suit = []
reasons_forecast = []

for i in range(len(sarima_forecasted_temp_df)):
    forecast_temp = sarima_temperature_forecast[i]
    forecast_date = sarima_forecasted_temp_df['DATE'][i]
    suitable_crops, reasons = check_crop_suitability_sarima(forecast_temp, forecast_date, 제주_crops)
    제주_suit.append(suitable_crops)
    suitable_crops_all, reasons = check_crop_suitability_sarima(forecast_temp, forecast_date, 제주_crops)
    전체_suit.append(suitable_crops_all)
    reasons_forecast.append(reasons)

제주_suit


[[], [], ['기타잡곡', '메밀', '무']]

In [117]:
전라남_crops = crop[crop['지역'] == '전라남도']

전라남_suit = []
reasons_forecast = []

for i in range(len(sarima_forecasted_temp_df)):
    forecast_temp = sarima_temperature_forecast[i]
    forecast_date = sarima_forecasted_temp_df['DATE'][i]
    suitable_crops, reasons = check_crop_suitability_sarima(forecast_temp, forecast_date, 전라남_crops)
    전라남_suit.append(suitable_crops)
    suitable_crops_all, reasons = check_crop_suitability_sarima(forecast_temp, forecast_date, 전라남_crops)
    전체_suit.append(suitable_crops_all)
    reasons_forecast.append(reasons)

전라남_suit


[['노지호박', '녹두', '미곡'],
 ['노지호박', '녹두', '미곡'],
 ['노지가을배추', '노지쪽파', '노지호박', '녹두', '미곡', '배', '배추', '팥', '호박']]

In [118]:
전라북_crops = crop[crop['지역'] == '전라북도']

전라북_suit = []
reasons_forecast = []

for i in range(len(sarima_forecasted_temp_df)):
    forecast_temp = sarima_temperature_forecast[i]
    forecast_date = sarima_forecasted_temp_df['DATE'][i]
    suitable_crops, reasons = check_crop_suitability_sarima(forecast_temp, forecast_date, 전라북_crops)
    전라북_suit.append(suitable_crops)
    suitable_crops_all, reasons = check_crop_suitability_sarima(forecast_temp, forecast_date, 전라북_crops)
    전체_suit.append(suitable_crops_all)
    reasons_forecast.append(reasons)

전라북_suit


[[], [], ['콩']]

In [119]:
전체_suit[0]


['노지참외', '수박', '오이']

In [120]:
전체_suit_월=[[],[],[]]
for i in range(len(전체_suit)):
    if i%3==0:
        전체_suit_월[0].append(전체_suit[i])
    elif i%3==1:
        전체_suit_월[1].append(전체_suit[i])
    else:
        전체_suit_월[2].append(전체_suit[i])
        
전체_suit_월[0]

[['노지참외', '수박', '오이'],
 [],
 ['건고추', '노지수박', '생강', '참외'],
 ['노지오이', '옥수수'],
 ['고추', '풋고추'],
 [],
 ['노지호박', '녹두', '미곡'],
 []]

In [121]:
전체_suit_월[1]

[['노지참외', '수박'],
 [],
 ['건고추', '노지수박', '생강'],
 ['노지오이', '옥수수'],
 ['고추', '풋고추'],
 [],
 ['노지호박', '녹두', '미곡'],
 []]

In [122]:
전체_suit_월[2]

[['가을일반무', '노지가을무', '노지참외', '수박'],
 ['가을총각무', '상추'],
 ['건고추', '노지수박', '사과', '생강'],
 ['옥수수', '잡곡'],
 ['고추', '노지시금치'],
 ['기타잡곡', '메밀', '무'],
 ['노지가을배추', '노지쪽파', '노지호박', '녹두', '미곡', '배', '배추', '팥', '호박'],
 ['콩']]

In [128]:
crop_substitution = {
    
    '가을일반무': ['노지가을무', '봄무', '가을총각무','고랭지무','노지겨울무','노지무','노지봄무','무','일반봄무','노지오이','오이','당근'],
    '노지가을무': ['가을일반무', '봄무', '가을총각무','고랭지무','노지겨울무','노지무','노지봄무','무','일반봄무','노지오이','오이','당근'],
    '봄무': ['노지가을무', '가을일반무', '가을총각무','고랭지무','노지겨울무','노지무','노지봄무','무','일반봄무','노지오이','오이','당근'],
    '가을총각무': ['노지가을무', '봄무', '가을일반무','고랭지무','노지겨울무','노지무','노지봄무','무','일반봄무','노지오이','오이','당근'],
    '고랭지무': ['노지가을무', '봄무', '가을총각무','가을일반무','노지겨울무','노지무','노지봄무','무','일반봄무','노지오이','오이','당근'],
    '노지겨울무': ['노지가을무', '봄무', '가을총각무','고랭지무','가을일반무','노지무','노지봄무','무','일반봄무','노지오이','오이','당근'],
    '노지무': ['노지가을무', '봄무', '가을총각무','고랭지무','노지겨울무','가을일반무','노지봄무','무','일반봄무','노지오이','오이','당근'],
    '노지봄무': ['노지가을무', '봄무', '가을총각무','고랭지무','노지겨울무','노지무','가을일반무','무','일반봄무','노지오이','오이','당근'],
    '무': ['노지가을무', '봄무', '가을총각무','고랭지무','노지겨울무','가을일반무','노지봄무','가을일반무','일반봄무','노지오이','오이','당근'],
    '일반봄무': ['노지가을무', '봄무', '가을총각무','고랭지무','노지겨울무','노지무','가을일반무','무','가을일반무','노지오이','오이','당근'],
    '노지오이': ['노지가을무', '봄무', '가을총각무','고랭지무','노지겨울무','노지무','노지봄무','무','일반봄무','가을일반무','오이','당근'],
    '오이': ['노지가을무', '봄무', '가을총각무','고랭지무','노지겨울무','노지무','노지봄무','무','일반봄무','노지오이','가을일반무','당근'],
    
    '당근': ['노지가을무', '봄무', '가을총각무','고랭지무','노지겨울무','노지무','노지봄무','무','일반봄무','노지오이','오이','가을일반무','호박','노지호박'],
    '호박': ['당근','노지호박'],
    '노지호박': ['당근','호박'],
    

    '건고추': ['고추', '풋고추'],
    '고추': ['건고추', '풋고추'],
    '풋고추': ['고추', '건고추'],
    
    '고랭지감자': ['봄감자'],
    '봄감자': ['고랭지감자'],

    '고랭지배추': ['노지가을배추','노지겨울배추','노지봄배추','노지배추','배추', '일반봄배추','노지상추','상추','시금치'],
    '노지가을배추': ['고랭지배추','노지겨울배추','노지봄배추','노지배추','배추', '일반봄배추','노지상추','상추','시금치'],
    '노지겨울배추': ['노지가을배추','고랭지배추','노지봄배추','노지배추','배추', '일반봄배추','노지상추','상추','시금치'],
    '노지봄배추': ['노지가을배추','노지겨울배추','고랭지배추','노지배추','배추', '일반봄배추','노지상추','상추','시금치'],
    '노지배추': ['노지가을배추','노지겨울배추','노지봄배추','고랭지배추','배추', '일반봄배추','노지상추','상추','시금치'],
    '배추': ['노지가을배추','노지겨울배추','노지봄배추','노지배추','고랭지배추', '일반봄배추','노지상추','상추','시금치'],
    '일반봄배추': ['노지가을배추','노지겨울배추','노지봄배추','노지배추','배추', '고랭지배추','노지상추','상추','시금치'],
    '노지상추': ['노지가을배추','노지겨울배추','노지봄배추','노지배추','배추', '일반봄배추','고랭지배추','상추','시금치'],
    '상추': ['노지가을배추','노지겨울배추','노지봄배추','노지배추','배추', '일반봄배추','노지상추','고랭지배추','시금치'],
    '시금치': ['노지가을배추','노지겨울배추','노지봄배추','노지배추','배추', '일반봄배추','노지상추','상추','고랭지배추'],

    '기타잡곡': ['잡곡', '옥수수','미곡','메밀','녹두','콩','팥'],
    '잡곡': ['기타잡곡', '옥수수','미곡','메밀','녹두','콩','팥'],
    '옥수수': ['잡곡', '기타잡곡','미곡','메밀','녹두','콩','팥'],
    '미곡': ['잡곡', '기타잡곡','옥수수','메밀','녹두','콩','팥'],
    '메밀': ['잡곡', '기타잡곡','옥수수','미곡','녹두','콩','팥'],
    '녹두': ['잡곡', '옥수수','미곡','메밀','기타잡곡','콩','팥'],
    '콩': ['잡곡', '옥수수','미곡','메밀','녹두','기타잡곡','팥'],
    '팥': ['잡곡', '옥수수','미곡','메밀','녹두','콩','기타잡곡'],


    '노지대파': ['대파', '파','노지쪽파','쪽파','노지파','양파','조미채소','마늘','생강'],
    '대파': ['노지대파', '파','노지쪽파','쪽파','노지파','양파''조미채소','마늘','생강'],
    '파': ['대파', '노지대파','노지쪽파','쪽파','노지파','양파''조미채소','마늘','생강'],
    '노지쪽파': ['대파', '파','노지대파','쪽파','노지파','양파''조미채소','마늘','생강'],
    '쪽파': ['대파', '파','노지쪽파','노지대파','노지파','양파''조미채소','마늘','생강'],
    '노지파': ['대파', '파','노지쪽파','쪽파','노지대파','양파''조미채소','마늘','생강'],
    '양파': ['대파', '파','노지쪽파','쪽파','노지파','노지대파''조미채소','마늘','생강'],
    '조미채소': ['대파', '파','노지쪽파','쪽파','노지파','양파','노지대파','마늘','생강'],
    '마늘': ['대파', '파','노지쪽파','쪽파','노지파','양파','조미채소','노지대파','생강'],
    '생강': ['대파', '파','노지쪽파','쪽파','노지파','양파','조미채소','마늘','노지대파'],
    
    
    '노지딸기': ['딸기', '노지참외','노지수박','배','사과','노지토마토','토마토'],
    '딸기': ['딸기', '노지딸기','노지수박','배','사과','노지토마토','토마토'],
    '노지참외': ['딸기', '노지딸기','노지수박','배','사과','노지토마토','토마토'],
    '노지수박': ['딸기', '노지참외','노지딸기','배','사과','노지토마토','토마토'],
    '배': ['딸기', '노지참외','노지수박','노지딸기','사과','노지토마토','토마토'],
    '사과': ['딸기', '노지참외','노지수박','배','노지딸기','노지토마토','토마토'],
    '노지토마토': ['딸기', '노지참외','노지수박','배','사과','노지딸기','토마토'],
    '토마토': ['딸기', '노지참외','노지수박','배','사과','노지토마토','노지딸기'],

}

In [130]:
def get_substitute_crops(crop_name):
    return crop_substitution.get(crop_name, [])

def check_crop_suitability(forecast_temp, forecast_date, crop_conditions):
    suitable_crops = []
    unsuitable_crops = []
    reasons = {}
    forecast_month = forecast_date.month
    for _, row in crop_conditions.iterrows():
        crop_name = row['작물명']
        planting_month = row['파종기']
        harvesting_month = row['수확기']
        
        reasons[crop_name] = []
        
        if not (planting_month - 1 <= forecast_month <= harvesting_month + 1):
            reasons[crop_name].append(f"Not within planting ({planting_month}) and harvesting ({harvesting_month}) period.")
        
        if not (row['생육기 최저 온도'] - 1 <= forecast_temp <= row['생육기 최고 온도'] + 1):
            reasons[crop_name].append(f"Temperature {forecast_temp:.2f}°C not in growing range ({row['생육기 최저 온도']}°C - {row['생육기 최고 온도']}°C).")
        
        if not reasons[crop_name]:
            suitable_crops.append(crop_name)
        else:
            unsuitable_crops.append(crop_name)
            reasons[crop_name] = ", ".join(reasons[crop_name])
    
    return suitable_crops, unsuitable_crops, reasons

suitable_crops_overall = []
unsuitable_crops_overall = []
reasons_forecast = []

for i in range(len(sarima_forecasted_temp_df)):
    forecast_temp = sarima_temperature_forecast[i]
    forecast_date = sarima_forecasted_temp_df['DATE'][i]
    suitable_crops, unsuitable_crops, reasons = check_crop_suitability(forecast_temp, forecast_date, crop)
    suitable_crops_overall.append(suitable_crops)
    unsuitable_crops_overall.append(unsuitable_crops)
    reasons_forecast.append(reasons)


alternative_crops_recommendations = []

for i, unsuitable_crops in enumerate(unsuitable_crops_overall):
    forecast_date = sarima_forecasted_temp_df['DATE'][i]
    forecast_temp = sarima_temperature_forecast[i]
    for crop_name in unsuitable_crops:
        substitutes = get_substitute_crops(crop_name)
        for substitute in substitutes:
            if substitute in suitable_crops_overall[i]:
                alternative_crops_recommendations.append({
                    'original_crop': crop_name,
                    'forecast_date': forecast_date,
                    'forecast_temp': forecast_temp,
                    'substitute_crop': substitute
                })
                break

alternative_crops_recommendations

[{'original_crop': '가을일반무',
  'forecast_date': Timestamp('2024-07-31 00:00:00'),
  'forecast_temp': 26.012077572561488,
  'substitute_crop': '노지오이'},
 {'original_crop': '가을총각무',
  'forecast_date': Timestamp('2024-07-31 00:00:00'),
  'forecast_temp': 26.012077572561488,
  'substitute_crop': '노지오이'},
 {'original_crop': '고랭지무',
  'forecast_date': Timestamp('2024-07-31 00:00:00'),
  'forecast_temp': 26.012077572561488,
  'substitute_crop': '노지오이'},
 {'original_crop': '기타잡곡',
  'forecast_date': Timestamp('2024-07-31 00:00:00'),
  'forecast_temp': 26.012077572561488,
  'substitute_crop': '옥수수'},
 {'original_crop': '노지가을무',
  'forecast_date': Timestamp('2024-07-31 00:00:00'),
  'forecast_temp': 26.012077572561488,
  'substitute_crop': '노지오이'},
 {'original_crop': '노지겨울무',
  'forecast_date': Timestamp('2024-07-31 00:00:00'),
  'forecast_temp': 26.012077572561488,
  'substitute_crop': '노지오이'},
 {'original_crop': '노지대파',
  'forecast_date': Timestamp('2024-07-31 00:00:00'),
  'forecast_temp': 26.0